# === Part I ===

In [5]:
!python count_freqs.py gene.train > gene.counts

# Fetch NGram Counts

In [6]:
import pandas as pd

df = pd.read_csv('gene.ngrams', sep=' ', names=['Count', 'NGram', 'One', 'Two', 'Three'])

# Unigram Tag Counts

In [7]:
counts = {}

for tag, count in df[df['NGram'] == '1-GRAM'][['One', 'Count']].values:
    counts[tag] = count

# Reference Counts

In [8]:
import pandas as pd

df = pd.read_csv('gene.counts', sep=' ', names=['Count', 'WORDTAG', 'Tag', 'Token'])

# Compile Token Counts

In [9]:
with open('gene.counts', 'r') as f:
    lines = f.readlines()

In [10]:
from collections import defaultdict

c = defaultdict(lambda: defaultdict(int))

for line in lines:
    count, _, tag, token = line.split()
    c[token][tag] = int(count)

# Find Infrequent Words

In [11]:
infrequent_words = set()

for token in c:
    if c[token]['O'] + c[token]['I-GENE'] < 5:
        infrequent_words.add(token)

# Write Infrequent Words Back To Disk

In [12]:
with open('gene.count+infrequents', 'w') as f:
    for line in lines:
        count, WORDTAG, tag, token = line.split()
        
        if token in infrequent_words:
            f.write(' '.join([count, WORDTAG, tag, '_RARE_']) + '\n')
        else:
            f.write(line)

# Recompute Counts with Infrequent Word Class

In [13]:
with open('gene.count+infrequents', 'r') as f:
    lines = f.readlines()  

In [14]:
c = defaultdict(lambda: defaultdict(int))

for line in lines:
    count, _, tag, token = line.split()
    c[token][tag] += int(count)

# Append Infrequent Word Count

In [15]:
with open('gene.counts', 'a') as f:
    for tag in c['_RARE_']:
        f.write(' '.join([str(c['_RARE_'][tag]), 'WORDTAG', tag, '_RARE_']) + '\n')

# Read All Words Back in One More Time

In [16]:
with open('gene.counts', 'r') as f:
    lines = f.readlines()

In [17]:
from collections import defaultdict

for line in lines:
    count, _, tag, token = line.split()
    c[token][tag] = int(count)

# Compute Emission Probabilities

In [18]:
e = defaultdict(lambda: defaultdict(int))

for token in c:
    for tag in c[token]:
        e[token][tag] = c[token][tag] / float(counts[tag])

# === Part II ===

In [19]:
import pandas as pd

df = pd.read_csv('gene.ngrams', sep=' ', names=['Count', 'NGram', 'One', 'Two', 'Three'])

# Bigrams

In [20]:
from collections import defaultdict

bigram_count = defaultdict(lambda: defaultdict(int))

for count, BIGRAM, tag_one, tag_two, _ in df[df['NGram'] == '2-GRAM'].values.tolist():
    bigram_count[tag_one][tag_two] = count

# Trigrams

In [21]:
from collections import defaultdict

trigram_count = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for count, TRIGRAM, tag_one, tag_two, tag_three in df[df['NGram'] == '3-GRAM'].values.tolist():
    trigram_count[tag_one][tag_two][tag_three] = count

# Compute $q_\text{MLE}$

In [22]:
import itertools
from collections import defaultdict

q = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for next, current, previous in itertools.product(['*', 'STOP', 'O', 'I-GENE'], repeat=3):
    # Ignore situations where the premise is impossible
    if not bigram_count[previous][current]:
        continue
        
    q[next][previous][current] = trigram_count[previous][current][next] / float(bigram_count[previous][current])

# Viterbi Algorithm

In [88]:
import itertools
from collections import defaultdict

pi = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


def viterbi(xs, q, e):
    # Initialize pi
    pi[0]['*']['*'], n = 1, len(xs)
    
    # Initialize S such that:
    #   S(-1) = S(0) = ['*']
    #   S(1:n) (inclusive) = ['O', 'I-GENE']
    #   S(n+1) = ['STOP']
    S = [['*']] + [('O', 'I-GENE') for _ in range(n) ] + [['STOP']] + [['*']]
    
    for k, x in enumerate(xs.split(), start=1):
        for u, v in itertools.product(S[k-1], S[k]):
            print 'Calculating pi[{}][{}][{}] = max w in {}...'.format(k, u, v, S[k-2])
            
            pi[k][u][v] = max([ pi[k-1][w][u]*q[v][w][u]*e[x][v] for w in S[k-2] ])
        
    return max([ pi[n][u][v]*q['STOP'][u][v] for u, v in itertools.product(S[n-1], S[n]) ])

In [89]:
viterbi('the', q, e)

Calculating pi[1][*][O] = max w in ['*']...
Calculating pi[1][*][I-GENE] = max w in ['*']...


0.0

# Debugging Viterbi

In [198]:
import itertools
from collections import defaultdict

highest_prob, backpointer = defaultdict(lambda: defaultdict(lambda: defaultdict(int))), defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


def viterbi(xs, transition, emission):
    # Initialize highest_prob
    highest_prob[0]['*']['*'], n, = 1, len(xs.split())
    y = [''] * (n+1)
    
    # Initialize possible_tags such that:
    #   possible_tags(-1) = possible_tags(0) = ['*']
    #   possible_tags(1:n) (inclusive) = ['O', 'I-GENE']
    #   possible_tags(n+1) = ['STOP']
    possible_tags = [['*']] + [('O', 'I-GENE') for _ in range(n) ] + [['STOP']] + [['*']]
    
    print 'For k = 1...{}'.format(n)
    print
    
    for k, x in enumerate(xs.split(), start=1):
        print '** Time to compute highest_prob(k={}, u, v) for all u in possible_tags({})={} and v in possible_tags({})={}...'.format(k, k-1, possible_tags[k-1], k, possible_tags[k])
        print
        
        for u, v in itertools.product(possible_tags[k-1], possible_tags[k]):
            print '    ==========================================================================================================='
            print '    Calculating highest_prob(k={}, u={}, v={}) = max over w in possible_tags({})={} of highest_prob({}, w, u={}) * transition(v={} | w, u={}) * emission(x={} | v={})...'.format(k,u,v,k-2,possible_tags[k-2],k-1,u,v,u,x,v)
            print
            
            currents = []
            for w in possible_tags[k-2]:
                print '    Trying w={}...'.format(w)
                print '    Computing highest_prob({}, w={}, u={}) * transition(v={} | w={}, u={}) * emission(x={} | v={}) where:'.format(k-1,w,u,v,w,u,x,v)
                print
                print '        highest_prob({}, w={}, u={}) = {}'.format(k-1,w,u,highest_prob[k-1][w][u])
                print '        transition(v={} | w={}, u={}) = {}'.format(v,w,u,transition[v][w][u])
                print '        emission(x={} | v={}) = {}'.format(x,v,emission[x][v])
                print
                
                current = highest_prob[k-1][w][u] * transition[v][w][u] * emission[x][v]
                
                print '    Result = {}'.format(current)
                currents.append((current, w))
                print
               
            print
            print '    Highest probability tagging is: {}'.format(max(currents))
            print '    ==========================================================================================================='
            highest_prob[k][u][v], backpointer[k][u][v] = max(currents)
            
            print       
    
    print '** Finally compute max of highest_prob(n={}, u, v) * transition(STOP | u, v) over all u in possible_tags({})={} and v in possible_tags({})={}...'.format(n, n-1, possible_tags[n-1], n, possible_tags[n])
    print
    
    currents = []
    for u, v in itertools.product(possible_tags[n-1], possible_tags[n]):
        print '    ==========================================================================================================='
        print '    Computing highest_prob(n={}, u={}, v={}) * transition(STOP | u={}, v={}) where:'.format(n,u,v,u,v)
        print
        print '        highest_prob(n={}, u={}, v={}) = {}'.format(n,u,v,highest_prob[n][u][v])
        print '        transition(STOP | u={}, v={}) = {}'.format(u,v,transition['STOP'][u][v])
        print
        
        current = highest_prob[n][u][v] * transition['STOP'][u][v]
        print '    Result = {}'.format(current)
        currents.append((current, u, v))
       
    print
    print '    Highest probability tagging is: {}'.format(max(currents))
    print '    ==========================================================================================================='

    # Compute Backpoints
    _, y[n-1], y[n] = max(currents)
    for k in range(n-2, 0, -1):
        y[k] = backpointer[k+2][y[k+1]][y[k+2]]
        
    return y[1:]

In [200]:
viterbi('gene gene gene', q, e)

For k = 1...3

** Time to compute highest_prob(k=1, u, v) for all u in possible_tags(0)=['*'] and v in possible_tags(1)=('O', 'I-GENE')...

    Calculating highest_prob(k=1, u=*, v=O) = max over w in possible_tags(-1)=['*'] of highest_prob(0, w, u=*) * transition(v=O | w, u=*) * emission(x=gene | v=O)...

    Trying w=*...
    Computing highest_prob(0, w=*, u=*) * transition(v=O | w=*, u=*) * emission(x=gene | v=O) where:

        highest_prob(0, w=*, u=*) = 1
        transition(v=O | w=*, u=*) = 0.945708901131
        emission(x=gene | v=O) = 0.00165445863564

    Result = 0.00156463625827


    Highest probability tagging is: (0.0015646362582742211, '*')

    Calculating highest_prob(k=1, u=*, v=I-GENE) = max over w in possible_tags(-1)=['*'] of highest_prob(0, w, u=*) * transition(v=I-GENE | w, u=*) * emission(x=gene | v=I-GENE)...

    Trying w=*...
    Computing highest_prob(0, w=*, u=*) * transition(v=I-GENE | w=*, u=*) * emission(x=gene | v=I-GENE) where:

        highest_prob(0

['I-GENE', 'I-GENE', 'O']